In [ ]:
from extra_data import RunDirectory 
import h5py 
import numpy as np 
%matplotlib widget
import matplotlib.pyplot as plt 

In [ ]:
def extract_gifs(r, verbose=True):
    run = RunDirectory(path=f'/gpfs/exfel/exp/SPB/202202/p003046/raw/r%04d' % (r))  
    
    # The require_all=True ensures we only get trains which exist for all sources
    sel = run.select([
        ('SPB_XTD9_XGM/XGM/DOOCS:output', 'data.intensitySa1TD'),
        ('SPB_IRU_MOTORS/MDL/DATA_SELECT', '*'),
        ('SPB_EXP_ZYLA/CAM/1:daqOutput', 'data.image.pixels'),
        ('LA1_LAS_PPL/CTRL/TRGDLY', 'triggerFE_UPSTREAM.value'), 
    ], require_all=True)
    if(verbose):
        sel.info()   
    sidemic_extra = sel['SPB_EXP_ZYLA/CAM/1:daqOutput', 'data.image.pixels']
    trains = sidemic_extra.train_id_coordinates()
    sidemic = sidemic_extra.xarray()
    extra = sel['LA1_LAS_PPL/CTRL/TRGDLY', 'triggerFE_UPSTREAM.value']
    delay = extra.xarray()
    return sidemic, delay

In [ ]:
sidemic, delay = extract_gifs(188)

In [ ]:
xray_delay = delay[0]

In [ ]:
dt = [0, 1, 2, 7.2, 31, 95]
tol_dt = [0.05, 0.05, 0.05, 0.08, 1, 1]

In [ ]:
runs = [188, 190, 192]

In [ ]:
runs = [190]

In [ ]:
cmap = 'gray'
for r in runs:
    #sidemic, delay = extract_gifs(r)
    for t, tol_t in zip(dt, tol_dt):
        idx = np.where(np.abs((delay-xray_delay) - t) <  tol_t)[0]
        if(len(idx) == 0):
            continue
        idx = np.min(idx).astype(int)+160
        fig = plt.figure(figsize=(1,2.7),dpi=400, frameon=False)
        ax = plt.Axes(fig, [0., 0., 1., 1.])
        ax.set_axis_off()
        fig.add_axes(ax)
        img = plt.imshow(sidemic[idx,200:1280,200:600])
        img.set_cmap(cmap)
        print('Time Delay = %.1f us' % ((delay[idx]-xray_delay)))
        plt.savefig('../figures/LJ_Fig3_delta_T_%.1fus_%s.png' % ((delay[idx]-xray_delay), cmap))

In [ ]:
plt.figure()
plt.plot(delay-xray_delay)